# Today's cryptocurrencies historical data

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pendulum
from sqlalchemy import create_engine
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
import os
import requests
import json

#### Context : Finance

#### Goal: Our goal is to create tables showing today’s 10 best/most profitable cryptocurrencies to invest in. We will look at historical data for the previous year and create a database by hourly and daily price changes. 

#### METHOD: we aim to do this by extracting data based on the ranks of the cryptocurrency and also the daily and hourly change of the cryptocurrency. 

#### FINDING DATA: Following are the sources for finding and collecting our data. Sources: Cryptocurrencies: https://coinmarketcap.com/ 
CSV files: https://www.cryptodatadownload.com/data/binance/
APIs: https://financialmodelingprep.com/api/v3/ https://rest.coinapi.io/v1/

#### Data collection tools: Splinter APIs BeautifulSoup

#### DATA AND CLEANUP: 
##### Cleanup process: Pandas SqlAlchemy Database: PostgreSQL

##### TEAM MEMBERS Juan Castaneda Elif Evrim Polat Nichole Edet

<div><h3 style="color:green;">Web Scraping - Obtaining today's best cryptocurrencies liste at https://coinmarketcap.com </h3></div>

In [2]:
# Obtaining Tables at the next link using pandas
url = 'https://coinmarketcap.com/'

In [3]:
tables = pd.read_html(url)

In [4]:
df = tables[0]
df.head()

,Unnamed: 0,#,Name,Price,24h %,7d %,Market Cap,Volume(24h),Circulating Supply,Last 7 Days,Unnamed: 10
0,NaN,1.0,Bitcoin1BTCBuy,"$62,738.28",0.71%,12.05%,"$1,172,118,943,649","$76,331,263,1661,216,662 BTC","18,682,675 BTC",NaN,NaN
1,NaN,2.0,Ethereum2ETHBuy,"$2,390.12",3.76%,20.97%,"$275,986,807,957","$33,962,247,95914,209,415 ETH","115,469,714 ETH",NaN,NaN
2,NaN,3.0,Binance Coin3BNBBuy,$542.46,3.55%,45.55%,"$83,828,159,781","$7,372,647,05013,591,085 BNB","154,532,785 BNB",NaN,NaN
3,NaN,4.0,XRP4XRP,$1.77,2.79%,96.30%,"$80,310,392,995","$29,098,995,91816,451,315,886 XRP","45,404,028,640 XRP",NaN,NaN
4,NaN,5.0,Cardano5ADA,$1.45,2.44%,22.90%,"$46,299,662,439","$9,445,003,1416,517,366,804 ADA","31,948,309,441 ADA",NaN,NaN


In [5]:
# using only the tables that will match our PostgreSQL database schema
columns = ['#', 'Name', 'Price']
cryptos_df = df[columns]
cryptos_df.head()

,#,Name,Price
0,1.0,Bitcoin1BTCBuy,"$62,738.28"
1,2.0,Ethereum2ETHBuy,"$2,390.12"
2,3.0,Binance Coin3BNBBuy,$542.46
3,4.0,XRP4XRP,$1.77
4,5.0,Cardano5ADA,$1.45


In [6]:
# converting # column values to integer
cryptos_df['#'] = cryptos_df['#'].astype(float).map("{:.0f}".format).copy()
cryptos_df.rename(columns={'#':'id','Name':'coin_name','Price':'latest_price'}, inplace=True)
# obtaiting only the first 10 coins
cryptos_df = cryptos_df[:10]

In [7]:
cryptos_df.head()

,id,coin_name,latest_price
0,1,Bitcoin1BTCBuy,"$62,738.28"
1,2,Ethereum2ETHBuy,"$2,390.12"
2,3,Binance Coin3BNBBuy,$542.46
3,4,XRP4XRP,$1.77
4,5,Cardano5ADA,$1.45


In [8]:
# Splitting Name column into 2 different columns which will containt the coin name and coin symbol using a number as a delimiter
# and using n=1 to do this only once, in case the coin name contains a number already
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
columnsplit.head()

,0,1,2
0,Bitcoin,1,BTCBuy
1,Ethereum,2,ETHBuy
2,Binance Coin,3,BNBBuy
3,XRP,4,XRP
4,Cardano,5,ADA


In [9]:
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
# since coin_symbol column originally contains a recommendation to "Buy" if applicable on coinmarketcap, we will get rid of that
# columnsplit[1] = columnsplit[1].str.replace('Buy', '', regex=True) by ignoring index 1
cryptos_df = cryptos_df.assign(coin_name=columnsplit[0],coin_symbol=columnsplit[2].str.replace('Buy',''))

In [10]:
cryptos_df

,id,coin_name,latest_price,coin_symbol
0,1,Bitcoin,"$62,738.28",BTC
1,2,Ethereum,"$2,390.12",ETH
2,3,Binance Coin,$542.46,BNB
3,4,XRP,$1.77,XRP
4,5,Cardano,$1.45,ADA
5,6,Tether,$1.00,USDT
6,7,Polkadot,$41.88,DOT
7,8,Uniswap,$34.96,UNI
8,9,Litecoin,$269.21,LTC
9,10,Chainlink,$40.13,LINK
